Data for the fingerprints depending on latitud and longitud

In [2]:
function get_fingerprints()

    fp_dir = joinpath(@__DIR__, "..", "data")
    isdir(fp_dir) || mkpath(fp_dir)
    fp_file = joinpath(fp_dir, "FINGERPRINTS_SLANGEN_Bakker.nc")
    if !isfile(fp_file)
        url = "https://github.com/scrim-network/BRICK/raw/master/fingerprints/FINGERPRINTS_SLANGEN_Bakker.nc"
        download(url, fp_file)
    end

    fplat = ncread(fp_file,"lat")
    fplon = ncread(fp_file,"lon")
    fpAIS = ncread(fp_file,"AIS")
    fpGSIC = ncread(fp_file,"GLAC")
    fpGIS = ncread(fp_file,"GIS")
    ncclose()

    return fplat,fplon,fpAIS,fpGSIC,fpGIS
end

get_fingerprints (generic function with 1 method)

In [12]:
using NetCDF
using DataFrames
(fplat,fplon,fpAIS,fpGSIC,fpGIS) = get_fingerprints()
DataFrame(A=[fplat],B=[fplon])

Row,A,B
,Array…,Array…
1,"[-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5, -81.5, -80.5 … 80.5, 81.5, 82.5, 83.5, 84.5, 85.5, 86.5, 87.5, 88.5, 89.5]","[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5 … 350.5, 351.5, 352.5, 353.5, 354.5, 355.5, 356.5, 357.5, 358.5, 359.5]"


The matrices of fpAIS, fpGSIC, and fpGIS, are constructed in the form of [Lon,Lat], therefore we want to nearest to or segment. The functions minimumlat(x) and minimumlon(x) help us to find this values.

In [49]:
function minimumlat(x)
    Distance=zeros(1,180);
    for i in 1:180
        distance1=(x-fplat[i])^2;
        Distance[i]=distance1;
    end
    m=minimum(Distance)
    RowNumber=findall(r->r==m,Distance);
    return(RowNumber)
end

minimumlat (generic function with 1 method)

In [54]:
function minimumlon(x)
    Distance=zeros(1,360);
    for i in 1:360
        distance1=(x-fplon[i])^2;
        Distance[i]=distance1;
    end
    m=minimum(Distance)
    RowNumber=findall(r->r==m,Distance);
    return(RowNumber)
end

minimumlon (generic function with 1 method)

Segments selected Manzanillo,Colima : lat 19.1138 lon -104.3385
                  New Orlands: lat 29.9511 lon -90.0715

In [52]:
#Latitude selected for the segment
minimumlat(19.1138)


1-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 110)

In [56]:
#Longitud selected for the sectment
lon1=mod(-104.3385,360)
minimumlon(lon1)

1-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 256)

Fingerprint value

In [62]:
fpAIS[256,110]*AIS+fpGIS[256,110]*GIS+fpGLAC*GLAC+TE+LWS

UndefVarError: UndefVarError: AIS_RCP not defined

Code to obtain the components of global sea level rise

In [79]:
function get_brickGMSL_zip(gmslfile::String, rcp::Union{String, Number})

    results_dir = joinpath(@__DIR__, "..", "data", "lslr")
    filepath_AIS = joinpath(results_dir,"projections_csv/RCP$(rcp)/projections_antarctic_RCP$(rcp)_sneasybrick.csv")
    filepath_GSIC = joinpath(results_dir,"projections_csv/RCP$(rcp)/projections_glaciers_RCP$(rcp)_sneasybrick.csv")
    filepath_GIS = joinpath(results_dir,"projections_csv/RCP$(rcp)/projections_greenland_RCP$(rcp)_sneasybrick.csv")
    filepath_TE = joinpath(results_dir,"projections_csv/RCP$(rcp)/projections_thermal_RCP$(rcp)_sneasybrick.csv")
    filepath_GMSL = joinpath(results_dir,"projections_csv/RCP$(rcp)/projections_gmsl_RCP$(rcp)_sneasybrick.csv")
    filepath_LWS = joinpath(results_dir,"projections_csv/RCP$(rcp)/projections_landwater_storage_sl_RCP$(rcp)_sneasybrick.csv")
    filepath_MAP = joinpath(results_dir,"projections_csv/RCP$(rcp)/projections_MAP_RCP$(rcp)_sneasybrick.csv")

    if !isfile(filepath_AIS) | !isfile(filepath_GSIC) | !isfile(filepath_GIS) |
       !isfile(filepath_TE) | !isfile(filepath_GMSL) | !isfile(filepath_LWS) |
       !isfile(filepath_MAP)
        url = "https://zenodo.org/record/6626335/files/sneasybrick_projections_csv.zip"
        download(url, gmslfile)
        run(pipeline(`unzip $(gmslfile) -d $(results_dir)`));
    end

    brAIS = Matrix(CSV.read(filepath_AIS,DataFrame))
    brGSIC = Matrix(CSV.read(filepath_GSIC,DataFrame))
    brGIS = Matrix(CSV.read(filepath_GIS,DataFrame))
    brTE = Matrix(CSV.read(filepath_TE,DataFrame))
    brGMSL = Matrix(CSV.read(filepath_GMSL,DataFrame))
    brLWS = Matrix(CSV.read(filepath_LWS,DataFrame))
    brMAP = CSV.read(filepath_MAP,DataFrame)
    btime = brMAP[!,:YEAR]

    return btime, brAIS, brGSIC, brGIS, brTE, brLWS, brGMSL

end

get_brickGMSL_zip (generic function with 2 methods)

In [80]:
get_brickGMSL_zip(gmslfile::String, local::rcp::Union{String, Number})



ErrorException: syntax: invalid "::" syntax around /Users/ce3304/Library/CloudStorage/GoogleDrive-ce3304@g.rit.edu/My Drive/Tutorials/Tutorials/CIAMPractice/FingerPrint.ipynb:1

In [14]:
using NetCDF
function downscale_brick(brickcomps, lonlat, ensInds, ystart=2010, yend=2100, tstep=10)
    # To do - check with vectors of lat, lon
    (fplat,fplon,fpAIS,fpGSIC,fpGIS) = get_fingerprints()
    (btime,AIS,GSIC,GIS,TE,LWS,GMSL) = brickcomps

    # Select indices of time of interest, with respect to timestep
    tinds = findall(x -> x .>= ystart && x .<=yend, btime)
    years = collect(ystart:yend)
    yinds = findall(x -> x % tstep==0, years)
    # Need to normalize LSL relative to 2000
    inorm = findall(x -> x==2000, btime)

    tdim=length(btime)

    if length(years)==length(tinds)
        tinds = tinds[yinds]
    else
        println("Error: years outside of bounds")
        return nothing
    end

    num_ens = length(ensInds)

    # Output matrix: ens x time x segment
    lsl_out = zeros(num_ens, length(tinds), length(lonlat))

    # Trim component vectors to timesteps and ensembles. Assume interval is 1 year
    if tdim==size(AIS)[1] # check that time dimension is 1
        # for normalizing
        AIS_norm = AIS[inorm,ensInds]
        GSIC_norm = GSIC[inorm,ensInds]
        GIS_norm = GIS[inorm,ensInds]
        TE_norm = TE[inorm,ensInds]
        LWS_norm = LWS[inorm,ensInds]
        GMSL_norm = GMSL[inorm,ensInds]
        # actual projections
        AIS = AIS[tinds,ensInds]
        GSIC = GSIC[tinds,ensInds]
        GIS = GIS[tinds,ensInds]
        TE = TE[tinds,ensInds]
        LWS = LWS[tinds,ensInds]
        GMSL = GMSL[tinds,ensInds]
    else
        println("Error: time dimension is not 1 for brick components")
        return nothing
    end

    for f in 1:length(lonlat) # Loop through lonlat tuples

        lon = lonlat[f][1]
        lat = lonlat[f][2]
        # Convert Longitude to degrees East
        # CIAM Lat is already in (-90,90) by default
        if lon <0
            lon = lon + 360
        end

        # Find fingerprint degrees nearest to lat,lon
        ilat = findall(isequal(minimum(abs.(fplat.-lat))),abs.(fplat.-lat))
        ilon = findall(isequal(minimum(abs.(fplon.-lon))),abs.(fplon.-lon))


        # Take average of closest lat/lon values
        fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[ilon,ilat])))
        fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[ilon,ilat])))
        fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[ilon,ilat])))

        fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
        fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
        fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]
        fpTE_loc = 1.0
        fpLWS_loc=1.0

        # Keep searching nearby lat/lon values if fingerprint value is NaN unless limit is hit
        inc =1

        while isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc) && inc<5

            newlonStart = lon_subtractor.(fplon[ilon],inc)[1]
            newlatStart = lat_subtractor.(fplat[ilat],inc)[1]
            newlonEnd = lon_adder.(fplon[ilon],inc)[1]
            newlatEnd = lat_adder.(fplat[ilat],inc)[1]

            latInd1 = minimum(findall(isequal(minimum(abs.(fplat.-newlatStart))),abs.(fplat.-newlatStart)))
            #minimum(findall(x-> x in newlatStart,fplat))
            latInd2 = maximum(findall(isequal(minimum(abs.(fplat.-newlatEnd))),abs.(fplat.-newlatEnd)))
            #maximum(findall(x -> x in newlatEnd,fplat))

            lonInd1 = minimum(findall(isequal(minimum(abs.(fplon.-newlonStart))),abs.(fplon.-newlonStart)))
            #minimum(findall(x-> x in newlonStart,fplon))
            lonInd2 = maximum(findall(isequal(minimum(abs.(fplon.-newlonEnd))),abs.(fplon.-newlonEnd)))
            #maximum(findall(x -> x in newlonEnd,fplon))

            if latInd2 < latInd1
                latInds=[latInd1; 1:latInd2]
            else
                latInds=latInd1:latInd2
            end

            if lonInd2 < lonInd1
                lonInds=[lonInd1; 1:lonInd2]
            else
                lonInds = lonInd1:lonInd2
            end

            fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[lonInds,latInds])))
            fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[lonInds,latInds])))
            fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[lonInds,latInds])))

            fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
            fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
            fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]

            inc = inc + 1

        end

        # If still NaN, throw an error
        if isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc)
            println("Error: no fingerprints found for ($(lon),$(lat))")
            return nothing
        end

       # Multiply fingerprints by BRICK ensemble members
       if ndims(AIS) > 1
            for n in 1:size(AIS)[2] # loop through ensemble members
                lsl_out[n, :, f] = fpGIS_loc * GIS[:,n] + fpAIS_loc * AIS[:,n] + fpGSIC_loc * GSIC[:,n] +
                                   fpTE_loc * TE[:,n] + fpLWS_loc * LWS[:,n]
                # CIAM - LSL should be sea-level change relative to year 2000
                lsl_norm = fpGIS_loc * GIS_norm[n] + fpAIS_loc * AIS_norm[n] + fpGSIC_loc * GSIC_norm[n] +
                           fpTE_loc * TE_norm[n] + fpLWS_loc * LWS_norm[n]
                lsl_out[n, :, f] = lsl_out[n, :, f] .- lsl_norm
            end
        else
            lsl_out[1, :, f] = fpGIS_loc * GIS[:] + fpAIS_loc * AIS[:] + fpGSIC_loc * GSIC[:] +
                fpTE_loc * TE[:] + fpLWS_loc * LWS[:]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm + fpAIS_loc * AIS_norm + fpGSIC_loc * GSIC_norm +
                       fpTE_loc * TE_norm + fpLWS_loc * LWS_norm
            lsl_out[1, :, f] = lsl_out[1, :, f] .- lsl_norm
        end

    end # End lonlat tuple

    return lsl_out,GMSL
end

downscale_brick (generic function with 4 methods)

In [15]:
downscale_brick(brickcomps, lonlat, ensInds, ystart=2010, yend=2100, tstep=10)

UndefVarError: UndefVarError: brickcomps not defined